# **CONSTRUCCIÓN DE MATRIZ DE COVARIABLES**


## 1. Carga de datos e importación de Bibliotecas

In [1]:
import os
import gc
import warnings

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
register_matplotlib_converters()
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Carga de datos
sales = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Proyecto Integrador/sales_train_validation.csv')#Continene los datos historicos de la venta diaria de cada uno de los productos
submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Proyecto Integrador/sample_submission.csv') #Ejemplo del archivo que hay que mandar
cal = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Proyecto Integrador/calendar.csv')#Contiene información de días en los que se llevaron a cabo promociones y campañas
prices = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Proyecto Integrador/sell_prices.csv')# Contiene información del precio de los productos por dia y por tienda

In [0]:
#print(sales.head())
#print(submission.head())
#print(cal.tail(70))
#print(prices.head())

In [0]:
#Filtrar para referencia, categoria, estado, tienda, etc
sales = sales[sales.store_id.isin(['WI_3'])]
sales = sales[sales.cat_id.isin(['HOUSEHOLD'])]
prices = prices[prices.store_id.str.contains('WI_3')]
prices = prices[prices.item_id.str.contains('HOUSEHOLD')]
#submission= submission[submission.id.isin(['FOODS_3_555_CA_1_validation'])]
submission = submission[submission.id.str.contains('WI_3')]
submission = submission[submission.id.str.contains('HOUSEHOLD')]
submission = submission[submission.id.str.contains('validation')]

In [0]:
#print(sales.head())
#print(submission.tail())
#print(cal.head())
#print(prices.head())

In [7]:
print(sales.shape)
print(prices.shape)
print(submission.shape)

(1047, 1919)
(238104, 4)
(1047, 29)


In [0]:
#Delimitación de número de datos a utilizar
sales.drop(sales.columns[6:1550], axis=1, inplace=True)

In [9]:
sales.head()

id          item_id      dept_id  \
28006  HOUSEHOLD_1_001_WI_3_validation  HOUSEHOLD_1_001  HOUSEHOLD_1   
28007  HOUSEHOLD_1_002_WI_3_validation  HOUSEHOLD_1_002  HOUSEHOLD_1   
28008  HOUSEHOLD_1_003_WI_3_validation  HOUSEHOLD_1_003  HOUSEHOLD_1   
28009  HOUSEHOLD_1_004_WI_3_validation  HOUSEHOLD_1_004  HOUSEHOLD_1   
28010  HOUSEHOLD_1_005_WI_3_validation  HOUSEHOLD_1_005  HOUSEHOLD_1   

          cat_id store_id state_id  d_1545  d_1546  d_1547  d_1548  d_1549  \
28006  HOUSEHOLD     WI_3       WI       0       0       0       0       0   
28007  HOUSEHOLD     WI_3       WI       0       0       0       0       0   
28008  HOUSEHOLD     WI_3       WI       0       0       2       0       0   
28009  HOUSEHOLD     WI_3       WI       0       0       0       0       1   
28010  HOUSEHOLD     WI_3       WI       0       2       0       1       1   

       d_1550  d_1551  d_1552  d_1553  d_1554  d_1555  d_1556  d_1557  d_1558  \
28006       0       0       0       0       0       0       0       0       0   
28007       0       0       0       0       0       0       0       0       0   
28008       0       0       0       0       0       0       3       0       0   
28009       0       5       2       0       0       0       0       1       0   
28010       1       0       0       0       0       0       1       1       0   

       d_1559  d_1560  d_1561  d_1562  d_1563  d_1564  d_1565  d_1566  d_1567  \
28006       0       0       0       0       0       0       0       0       0   
28007       0       0       0       0       0       0       0       0       0   
28008       0       0       0       0       0       0       2       0       2   
28009       0       0       0       2       1       1       1       1       2   
28010       1       0       0       2       0       1       2       1       0   

       d_1568  d_1569  d_1570  d_1571  d_1572  d_1573  d_1574  d_1575  d_1576  \
28006       0       0       0       0       0       0       0       0       0   
28007       0       0       0       0       0       0       0       0       2   
28008       0       0       0       0       0       0       3       0       0   
28009       3       0       8       0       2       4       0       0       0   
28010       0       0       1       0       1       0       0       2       0   

       d_1577  d_1578  d_1579  d_1580  d_1581  d_1582  d_1583  d_1584  d_1585  \
28006       0       0       0       0       0       0       0       0       0   
28007       0       0       0       0       1       3       0       0       1   
28008       0       0       0       0       0       1       0       2       1   
28009       3       0       1       1       1       1       5       0       1   
28010       2       1       0       1       3       3       0       1       2   

       d_1586  d_1587  d_1588  d_1589  d_1590  d_1591  d_1592  d_1593  d_1594  \
28006       0       0       0       0       0       0       0       0       0   
28007       0       1       0       1       3       0       1       0       1   
28008       0       0       0       0       1       1       0       0       1   
28009       0       2       0       3       4       1       0       2       2   
28010       1       1       1       1       2       1       2       1       0   

       d_1595  d_1596  d_1597  d_1598  d_1599  d_1600  d_1601  d_1602  d_1603  \
28006       0       0       0       0       0       0       0       0       0   
28007       1       0       0       0       0       0       0       1       0   
28008       0       0       0       1       1       0       0       0       0   
28009       1       3       3       0       0       0       0       0       0   
28010       1       0       2       0       1       0       1       1       1   

       d_1604  d_1605  d_1606  d_1607  d_1608  d_1609  d_1610  d_1611  d_1612  \
28006       0       0       0       0       0       0       0       0       0   
28007       1       0       0       0       1     

## 2. Enriquecer y hacer Doomies de matriz Calendario 

### 2.1 Agregar "Doomie" si hay evento hoy, mañana y pasado mañana en DF "calendar"

In [10]:
#HOY
#Creamos un vector booleano cuyo valor será 1 o 0 dependiendo si hay evento o no en el día (t)
hoyEvento = []
for i in cal['event_name_1']:
  if pd.isna(i) == True:
    hoyEvento.append("0")
  else:
    hoyEvento.append("1")

#No vale la pena tenerlo en cuenta porque hay muy pocos eventos 2 en todo el dataframe, solo 5.
#HOY EVENTO2 ---- Siempre que hay evento 2, hay evento 1, por lo tanto si este vale 1, es porque hay dos eventos el mismo día.
#hoyEvento2 = []
#for i in cal['event_name_2']:
  #if pd.isna(i) == True:
    #hoyEvento2.append("0")
  #else:
    #hoyEvento2.append("1")

# MAÑANA
#se crea un vector booleano empezando desde t+1, será 1 si hay evento en t+1 o 0 en caso de que no.
diaAnteriorEvento = []
for i in cal['event_name_1'][1:len(cal)]:
  if pd.isna(i) == True:
    diaAnteriorEvento.append("0")
  else:
    diaAnteriorEvento.append("1")

diaAnteriorEvento.append("0") 

#Este ultimo es para igualar el largo de las filas del DF Calendar. Recordar que el ciclo comenzó en el segundo elemento -->[1:len(calendar)]. Asumimos que para el ultimo día registrado en Calendar, el dia que le sigue no hubo evento. 
# PASADO MAÑANA
diaAnterior2Evento = []
for i in cal['event_name_1'][2:len(cal)]:
  if pd.isna(i) == True:
    diaAnterior2Evento.append("0")
  else:
    diaAnterior2Evento.append("1")

diaAnterior2Evento.append("0") 
diaAnterior2Evento.append("0")

len(hoyEvento)

1969

In [0]:
#Se agregan las nuevas variables de eventos al calendario 
cal["EventoHoy"]=hoyEvento
cal["EventoMañana"]=diaAnteriorEvento
cal["EventoPasadoMañana"]=diaAnterior2Evento
#cal["EventoHoy2"]=hoyEvento2              # No vale la pena tener en cuenta estos eventos ya que solo hay 5 en todo el dataset.

### 2.2 Agregar "Doomie" para event_type

In [12]:
Doomies_Event_Type=pd.get_dummies(cal["event_type_1"])
Doomies_Event_Type.rename(columns={'Cultural':'event_type_Bool_Cultural'}, inplace=True)
Doomies_Event_Type.rename(columns={'National':'event_type_Bool_National'}, inplace=True)
Doomies_Event_Type.rename(columns={'Religious':'event_type_Bool_Religious'}, inplace=True)
Doomies_Event_Type.rename(columns={'Sporting':'event_type_Bool_Sporting'}, inplace=True)
Doomies_Event_Type

event_type_Bool_Cultural  event_type_Bool_National  \
0                            0                         0   
1                            0                         0   
2                            0                         0   
3                            0                         0   
4                            0                         0   
...                        ...                       ...   
1964                         0                         0   
1965                         0                         0   
1966                         0                         0   
1967                         0                         0   
1968                         0                         0   

      event_type_Bool_Religious  event_type_Bool_Sporting  
0                             0                         0  
1                             0                         0  
2                             0                         0  
3                             0                         0  
4                             0                         0  
...                         ...                       ...  
1964                          0                         0  
1965                          0                         0  
1966                          0                         0  
1967                          0                         0  
1968                          0                         1  

[1969 rows x 4 columns]

In [13]:
# Se concatenan las doomies creadas y la tabla de calendario
cal = pd.concat([cal, Doomies_Event_Type], axis=1, sort=False)
cal

date  wm_yr_wk    weekday  wday  month  year       d  \
0     2011-01-29     11101   Saturday     1      1  2011     d_1   
1     2011-01-30     11101     Sunday     2      1  2011     d_2   
2     2011-01-31     11101     Monday     3      1  2011     d_3   
3     2011-02-01     11101    Tuesday     4      2  2011     d_4   
4     2011-02-02     11101  Wednesday     5      2  2011     d_5   
...          ...       ...        ...   ...    ...   ...     ...   
1964  2016-06-15     11620  Wednesday     5      6  2016  d_1965   
1965  2016-06-16     11620   Thursday     6      6  2016  d_1966   
1966  2016-06-17     11620     Friday     7      6  2016  d_1967   
1967  2016-06-18     11621   Saturday     1      6  2016  d_1968   
1968  2016-06-19     11621     Sunday     2      6  2016  d_1969   

      event_name_1 event_type_1  event_name_2 event_type_2  snap_CA  snap_TX  \
0              NaN          NaN           NaN          NaN        0        0   
1              NaN          NaN           NaN          NaN        0        0   
2              NaN          NaN           NaN          NaN        0        0   
3              NaN          NaN           NaN          NaN        1        1   
4              NaN          NaN           NaN          NaN        1        0   
...            ...          ...           ...          ...      ...      ...   
1964           NaN          NaN           NaN          NaN        0        1   
1965           NaN          NaN           NaN          NaN        0        0   
1966           NaN          NaN           NaN          NaN        0        0   
1967           NaN          NaN           NaN          NaN        0        0   
1968  NBAFinalsEnd     Sporting  Father's day     Cultural        0        0   

      snap_WI EventoHoy EventoMañana EventoPasadoMañana  \
0           0         0            0                  0   
1           0         0            0                  0   
2           0         0            0                  0   
3           0         0            0                  0   
4           1         0            0                  0   
...       ...       ...          ...                ...   
1964        1         0            0                  0   
1965        0         0            0                  0   
1966        0         0            0                  1   
1967        0         0            1                  0   
1968        0         1            0                  0   

      event_type_Bool_Cultural  event_type_Bool_National  \
0                            0                         0   
1                            0                         0   
2                            0                         0   
3                            0                         0   
4                            0                         0   
...                        ...                       ...   
1964                         0                         0   
1965                         0                         0   
1966                         0                         0   
1967                         0                         0   
1968                         0                         0   

      event_type_Bool_Religious  event_type_Bool_Sporting  
0                             0                         0  
1                             0                         0  
2                             0                         0  
3                             0                         0  
4                             0                         0  
...                         ...                       ...  
1964                          0                         0  
1965                          0                         0  
1966                          0                         0  
1967                          0                         0  
1968                          0                         1  

[1969 rows x 21 columns]

### 2.3 Definicion de funciones para creación de Variables día semana, numero día, semana, mes, trimestre y año

In [0]:
# Es importante saber si el hecho de que sea un día en particular de la semana, numero de día del mes, semana del año, mes, trimestre o año tiene
# relevancia en las predicciones. Por lo tanto se define una funcion para hacerlo.

def add_time_features(df, dt_col):
    df[dt_col] = pd.to_datetime(df[dt_col])

    attrs = [
        "quarter",
        #"week",
        #"year",
        #"day",
        #"month",
        #"dayofweek",
    ]

    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        df[attr] = getattr(df[dt_col].dt, attr).astype(dtype)

    df["is_weekend"] = df["wday"].isin([1, 2]).astype(np.int8)
    return df

In [0]:
cal = add_time_features(cal, "date")

In [0]:
cal["month"].replace({1: "01_January", 2: "02_February", 3: "03_March", 4: "04_April", 5: "05_May", 6: "06_June", 7: "07_July",8: "08_August",9: "09_September",10: "10_October",11: "11_November",12: "12_December"}, inplace=True)
cal["quarter"].replace({1: "quarter_1", 2: "quarter_2", 3: "quarter_3", 4: "quarter_4"}, inplace=True)
#cal["week"].replace({1: "week_01", 2: "week_02", 3: "week_03", 4: "week_04", 5: "week_05", 6: "week_06", 7: "week_07", 8: "week_08",9: "week_09",10: "week_10",11: "week_11", 12: "week_12", 13: "week_13", 14: "week_14", 15: "week_15", 16: "week_16", 17: "week_17", 18: "week_18",19: "week_19",20: "week_20",21: "week_21", 22: "week_22", 23: "week_23", 24: "week_24", 25: "week_25", 26: "week_26", 27: "week_27", 28: "week_28",29: "week_29",30: "week_30",31: "week_31", 32: "week_32", 33: "week_33", 34: "week_34", 35: "week_35", 36: "week_36", 37: "week_37", 38: "week_38",39: "week_39",40: "week_40",41: "week_41", 42: "week_42", 43: "week_43", 44: "week_44", 45: "week_45", 46: "week_46", 47: "week_47", 48: "week_48",49: "week_49",50: "week_50",51: "week_51", 52: "week_52", 53: "week_53"}, inplace=True)
#cal["day"].replace({1: "day_month_01", 2: "day_month_02", 3: "day_month_03", 4: "day_month_04", 5: "day_month_05", 6: "day_month_06", 7: "day_month_07", 8: "day_month_08",9: "day_month_09",10: "day_month_10",11: "day_month_11", 12: "day_month_12", 13: "day_month_13", 14: "day_month_14", 15: "day_month_15", 16: "day_month_16", 17: "day_month_17", 18: "day_month_18",19: "day_month_19",20: "day_month_20",21: "day_month_21", 22: "day_month_22", 23: "day_month_23", 24: "day_month_24", 25: "day_month_25", 26: "day_month_26", 27: "day_month_27", 28: "day_month_28",29: "day_month_29",30: "day_month_30",31: "day_month_31"}, inplace=True)
cal["year"].replace({2011: "year_2011", 2012: "year_2012", 2013: "year_2013", 2014: "year_2014", 2015: "year_2015",2016: "year_2016"}, inplace=True)
cal["weekday"].replace({"Monday": "01_Monday", "Tuesday": "02_Tuesday", "Wednesday": "03_Wednesday", "Thursday": "04_Thursday", "Friday": "05_Friday", "Saturday": "06_Saturday", "Sunday": "07_Sunday"}, inplace=True)

In [17]:
cal.head(10)

date  wm_yr_wk       weekday  wday        month       year     d  \
0 2011-01-29     11101   06_Saturday     1   01_January  year_2011   d_1   
1 2011-01-30     11101     07_Sunday     2   01_January  year_2011   d_2   
2 2011-01-31     11101     01_Monday     3   01_January  year_2011   d_3   
3 2011-02-01     11101    02_Tuesday     4  02_February  year_2011   d_4   
4 2011-02-02     11101  03_Wednesday     5  02_February  year_2011   d_5   
5 2011-02-03     11101   04_Thursday     6  02_February  year_2011   d_6   
6 2011-02-04     11101     05_Friday     7  02_February  year_2011   d_7   
7 2011-02-05     11102   06_Saturday     1  02_February  year_2011   d_8   
8 2011-02-06     11102     07_Sunday     2  02_February  year_2011   d_9   
9 2011-02-07     11102     01_Monday     3  02_February  year_2011  d_10   

  event_name_1 event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  \
0          NaN          NaN          NaN          NaN        0        0   
1          NaN          NaN          NaN          NaN        0        0   
2          NaN          NaN          NaN          NaN        0        0   
3          NaN          NaN          NaN          NaN        1        1   
4          NaN          NaN          NaN          NaN        1        0   
5          NaN          NaN          NaN          NaN        1        1   
6          NaN          NaN          NaN          NaN        1        0   
7          NaN          NaN          NaN          NaN        1        1   
8    SuperBowl     Sporting          NaN          NaN        1        1   
9          NaN          NaN          NaN          NaN        1        1   

   snap_WI EventoHoy EventoMañana EventoPasadoMañana  \
0        0         0            0                  0   
1        0         0            0                  0   
2        0         0            0                  0   
3        0         0            0                  0   
4        1         0            0                  0   
5        1         0            0                  0   
6        0         0            0                  1   
7        1         0            1                  0   
8        1         1            0                  0   
9        0         0            0                  0   

   event_type_Bool_Cultural  event_type_Bool_National  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   
5                         0                         0   
6                         0                         0   
7                         0                         0   
8                         0                         0   
9                         0                         0   

   event_type_Bool_Religious  event_type_Bool_Sporting    quarter  is_weekend  
0                          0                         0  quarter_1           1  
1                          0                         0  quarter_1           1  
2                          0                         0  quarter_1           0  
3                          0                         0  quarter_1           0  
4                          0                         0  quarter_1           0  
5                          0                         0  quarter_1           0  
6                          0                         0  quarter_1           0  
7                          0                         0  quarter_1           1  
8                          0                         1  quarter_1           1  
9                          0                         0  quarter_1           0

In [0]:
Doomies_weekday=pd.get_dummies(cal["weekday"])
Doomies_month=pd.get_dummies(cal["month"])
Doomies_year=pd.get_dummies(cal["year"])
#Doomies_week=pd.get_dummies(cal["week"])
Doomies_quarter=pd.get_dummies(cal["quarter"])
#Doomies_day=pd.get_dummies(cal["day"])

### 2.4 Matriz Final de Calendario

In [19]:
calendar = cal[{"date","wm_yr_wk"}]
calendar = pd.concat([calendar,cal.d], axis=1, sort=False)
calendar = pd.concat([calendar, Doomies_year], axis=1, sort=False)
calendar = pd.concat([calendar, Doomies_quarter], axis=1, sort=False)
calendar = pd.concat([calendar, Doomies_month], axis=1, sort=False)
#calendar = pd.concat([calendar, Doomies_week], axis=1, sort=False)
#calendar = pd.concat([calendar, Doomies_day], axis=1, sort=False)
calendar = pd.concat([calendar, Doomies_weekday], axis=1, sort=False)
calendar = pd.concat([calendar, cal.is_weekend, cal.EventoHoy, cal.EventoMañana, cal.EventoPasadoMañana, cal.event_type_Bool_Cultural, cal.event_type_Bool_National, cal.event_type_Bool_Religious, cal.event_type_Bool_Sporting], axis=1, sort=False)
calendar

wm_yr_wk       date       d  year_2011  year_2012  year_2013  year_2014  \
0        11101 2011-01-29     d_1          1          0          0          0   
1        11101 2011-01-30     d_2          1          0          0          0   
2        11101 2011-01-31     d_3          1          0          0          0   
3        11101 2011-02-01     d_4          1          0          0          0   
4        11101 2011-02-02     d_5          1          0          0          0   
...        ...        ...     ...        ...        ...        ...        ...   
1964     11620 2016-06-15  d_1965          0          0          0          0   
1965     11620 2016-06-16  d_1966          0          0          0          0   
1966     11620 2016-06-17  d_1967          0          0          0          0   
1967     11621 2016-06-18  d_1968          0          0          0          0   
1968     11621 2016-06-19  d_1969          0          0          0          0   

      year_2015  year_2016  quarter_1  quarter_2  quarter_3  quarter_4  \
0             0          0          1          0          0          0   
1             0          0          1          0          0          0   
2             0          0          1          0          0          0   
3             0          0          1          0          0          0   
4             0          0          1          0          0          0   
...         ...        ...        ...        ...        ...        ...   
1964          0          1          0          1          0          0   
1965          0          1          0          1          0          0   
1966          0          1          0          1          0          0   
1967          0          1          0          1          0          0   
1968          0          1          0          1          0          0   

      01_January  02_February  03_March  04_April  05_May  06_June  07_July  \
0              1            0         0         0       0        0        0   
1              1            0         0         0       0        0        0   
2              1            0         0         0       0        0        0   
3              0            1         0         0       0        0        0   
4              0            1         0         0       0        0        0   
...          ...          ...       ...       ...     ...      ...      ...   
1964           0            0         0         0       0        1        0   
1965           0            0         0         0       0        1        0   
1966           0            0         0         0       0        1        0   
1967           0            0         0         0       0        1        0   
1968           0            0         0         0       0        1        0   

      08_August  09_September  10_October  11_November  12_December  \
0             0             0           0            0            0   
1             0             0           0            0            0   
2             0             0           0            0            0   
3             0             0           0            0            0   
4             0             0           0            0            0   
...         ...           ...         ...          ...          ...   
1964          0             0           0            0            0   
1965          0             0           0            0            0   
1966          0             0           0            0            0   
1967          0             0           0            0            0   
1968          0             0           0            0            0   

      01_Monday  02_Tuesday  03_Wednesday  04_Thursday  05_Friday  \
0             0           0             0            0          0   
1             0           0             0            0          0   
2             1           0             0            0          0   
3             0           1             0            0          0 

## 3. Definición de Funciones.

###3.1 Despliegue de varios DF a la vez.

In [0]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df) #Para desplegar varios DF.

### 3.2 Optimización de uso de Memoria.

In [0]:
def reduce_mem_usage(df, verbose=False):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns

    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df


### 3.3 Funciones para creación de matriz de covariables a partir de la fusión de información de la referencia, el calendario y el precio

In [0]:
def extract_num(ser):
    return ser.str.extract(r"(\d+)").astype(np.int16) # Función para extraer el numero del día de las columna (d_1 ->1 // d_13 -> 13)


def reshape_sales(sales, submission, d_thresh=0, verbose=True):
    # Encabezados a los que se le aplicará método melt.
    id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

    # Dejar tabla sin la demanda de los días (Vamos a poner los días en fila y no columna.)
    product = sales[id_columns]

    
    sales = sales.melt(id_vars=id_columns, var_name="d", value_name="demand",)# Construir tabla con función melt para que la demanda me quede en función de los días en fila. 
    sales = reduce_mem_usage(sales) #Con esto me vuelve a quedar los días en columna (¿Para que lo devuelve?)

    # Separar el DataFrame "Submission" en validación y evaluación. 
    vals = submission[submission["id"].str.endswith("validation")]
    evals = submission[submission["id"].str.endswith("evaluation")]

    # Editar nombre de las columnas de los días para que quede acorde con lo que serán el reango de los días de predicción y el rango de los días de validación.
    vals.columns = ["id"] + [f"d_{d}" for d in range(1914, 1914 + DAYS_PRED)]
    evals.columns = ["id"] + [f"d_{d}" for d in range(1942, 1942 + DAYS_PRED)]

    # Unir tabla de validación y evaluación con la tabla de producto (item_id;dept_id;cat_id;store_id;state_id)
    evals["id"] = evals["id"].str.replace("_evaluation", "_validation")
    vals = vals.merge(product, how="left", on="id")
    evals = evals.merge(product, how="left", on="id")
    evals["id"] = evals["id"].str.replace("_validation", "_evaluation")


    #Impresión de tablas finales de validación y evaluación.
    if verbose:
        print("validation")
        display(vals)

        print("evaluation")
        display(evals)

    #Llevar estructura de la tabla de validación y evaluación a que los días esten en fila (La anterior la compusimos como lo pide submission que es los días en columna)
    vals = vals.melt(id_vars=id_columns, var_name="d", value_name="demand")
    evals = evals.melt(id_vars=id_columns, var_name="d", value_name="demand")

    #Agregar columna "part" para indicar si el dato corresponde a el dataset de entrenamiento, el de validación o el de evaluación. 
    sales["part"] = "train"
    vals["part"] = "validation"
    evals["part"] = "evaluation"

    #Unir todo en un solo dataset llamdo "data"
    data = pd.concat([sales, vals, evals], axis=0)

    #Eliminar los demas DF que ya se utilizaron para componer el DF data (Por memoria)
    del sales, vals, evals


    data["d"] = extract_num(data["d"])
    data = data[data["d"] >= d_thresh]

    # Elimianar DF "evals" de data (Por el momento)
    data = data[data["part"] != "evaluation"]

    gc.collect() #Garbage Colector

    if verbose:
        print("data")
        display(data)

    return data


def merge_calendar(data, calendar):
    #calendar = calendar.drop(["weekday", "wday", "month", "year"], axis=1) #Quitar Columnas "weekday", "wday", "month", "year" de Calendar // Despues se la añadiran con otros datos mas enriquecidos en la función add_time_features 
    return data.merge(calendar, how="left", on="d") #Unir tabla "data" con "calendar" a partir de la columna "d" (La que indica el día)


def merge_prices(data, prices):
    return data.merge(prices, how="left", on=["store_id", "item_id", "wm_yr_wk"]) #Unir tabla "data" con "prices" a partir de la correspondencia de las columnas "store_id", "item_id", "wm_yr_wk"

### 3.4 Función para enriquecer datos de sales y prices.

In [0]:
def add_demand_features(df): 

    #for window in [1]:
        #df[f"prom_ult_4_dias{window}"] = df.groupby(["id"])["demand"].transform(
            #lambda x: (x.shift(28)+x.shift(21)+x.shift(14)+x.shift(7))/4)
    
    for window in [7, 30, 60, 90, 180]:
        df[f"rolling_std_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(28).rolling(window).std()
        )

    for window in [7, 30, 60, 90, 180]:
        df[f"rolling_mean_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(28).rolling(window).mean()
        )

    #for window in [7, 30, 60]:
        #df[f"rolling_min_t{window}"] = df.groupby(["id"])["demand"].transform(
            #lambda x: x.shift(28).rolling(window).min()
        #)

    #for window in [7, 30, 60]:
        #df[f"rolling_max_t{window}"] = df.groupby(["id"])["demand"].transform(
            #lambda x: x.shift(28).rolling(window).max()
        #)

    df["rolling_skew_t30"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(28).rolling(30).skew()
    )
    df["rolling_kurt_t30"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(28).rolling(30).kurt()
    )
    return df


def add_price_features(df):
    df["shift_price_t1"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1)
    )
    df["price_change_t1"] = (df["shift_price_t1"] - df["sell_price"]) / (
        df["shift_price_t1"]
    )
    #df["rolling_price_max_t365"] = df.groupby(["id"])["sell_price"].transform(
        #lambda x: x.shift(1).rolling(365).max()
    #)
    #df["price_change_t365"] = (df["rolling_price_max_t365"] - df["sell_price"]) / (
        #df["rolling_price_max_t365"]
    #)

    df["rolling_price_std_t7"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(7).std()
    )
    df["rolling_price_std_t30"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(30).std()
    )
    return df.drop(["shift_price_t1"], axis=1)


## 4. Creación de matriz de covariables.

### 4.1 Selección de Referencia

In [0]:
NUM_ITEMS = sales.shape[0]  # 30490
DAYS_PRED = submission.shape[1] - 1  # 28

### 4.2 Unión de Matrices (sales + calendar + prices = data)

In [25]:
#data = reshape_sales(sales, submission, d_thresh=1941 - int(365 * 2))
data = reshape_sales(sales, submission, d_thresh=0)
del sales
gc.collect()

calendar["d"] = extract_num(calendar["d"]) # Extraer el numero del día de las columna (d_1 ->1 // d_13 -> 13).
data = merge_calendar(data, calendar) # Unir Dataframe de "calendar" con "data" con la función merge_calendar
del calendar 
gc.collect()

data = merge_prices(data, prices)# Unir Dataframe de "calendar" con "data" con la función merge_calendar
del prices
gc.collect()

data = reduce_mem_usage(data)

validation


id  d_1914  d_1915  d_1916  d_1917  d_1918  \
0  HOUSEHOLD_1_001_WI_3_validation       0       0       0       0       0   
1  HOUSEHOLD_1_002_WI_3_validation       0       0       0       0       0   
2  HOUSEHOLD_1_003_WI_3_validation       0       0       0       0       0   
3  HOUSEHOLD_1_004_WI_3_validation       0       0       0       0       0   
4  HOUSEHOLD_1_005_WI_3_validation       0       0       0       0       0   

   d_1919  d_1920  d_1921  d_1922  d_1923  d_1924  d_1925  d_1926  d_1927  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   d_1928  d_1929  d_1930  d_1931  d_1932  d_1933  d_1934  d_1935  d_1936  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   d_1937  d_1938  d_1939  d_1940  d_1941          item_id      dept_id  \
0       0       0       0       0       0  HOUSEHOLD_1_001  HOUSEHOLD_1   
1       0       0       0       0       0  HOUSEHOLD_1_002  HOUSEHOLD_1   
2       0       0       0       0       0  HOUSEHOLD_1_003  HOUSEHOLD_1   
3       0       0       0       0       0  HOUSEHOLD_1_004  HOUSEHOLD_1   
4       0       0       0       0       0  HOUSEHOLD_1_005  HOUSEHOLD_1   

      cat_id store_id state_id  
0  HOUSEHOLD     WI_3       WI  
1  HOUSEHOLD     WI_3       WI  
2  HOUSEHOLD     WI_3       WI  
3  HOUSEHOLD     WI_3       WI  
4  HOUSEHOLD     WI_3       WI

evaluation


Empty DataFrame
Columns: [d_1942, d_1943, d_1944, d_1945, d_1946, d_1947, d_1948, d_1949, d_1950, d_1951, d_1952, d_1953, d_1954, d_1955, d_1956, d_1957, d_1958, d_1959, d_1960, d_1961, d_1962, d_1963, d_1964, d_1965, d_1966, d_1967, d_1968, d_1969, id, item_id, dept_id, cat_id, store_id, state_id]
Index: []

data


id          item_id      dept_id     cat_id  \
0  HOUSEHOLD_1_001_WI_3_validation  HOUSEHOLD_1_001  HOUSEHOLD_1  HOUSEHOLD   
1  HOUSEHOLD_1_002_WI_3_validation  HOUSEHOLD_1_002  HOUSEHOLD_1  HOUSEHOLD   
2  HOUSEHOLD_1_003_WI_3_validation  HOUSEHOLD_1_003  HOUSEHOLD_1  HOUSEHOLD   
3  HOUSEHOLD_1_004_WI_3_validation  HOUSEHOLD_1_004  HOUSEHOLD_1  HOUSEHOLD   
4  HOUSEHOLD_1_005_WI_3_validation  HOUSEHOLD_1_005  HOUSEHOLD_1  HOUSEHOLD   

  store_id state_id     d  demand   part  
0     WI_3       WI  1545       0  train  
1     WI_3       WI  1545       0  train  
2     WI_3       WI  1545       0  train  
3     WI_3       WI  1545       0  train  
4     WI_3       WI  1545       0  train

### 4.3 Enriquecer matriz de covariables.

In [26]:
data = add_demand_features(data).pipe(reduce_mem_usage)
data = add_price_features(data).pipe(reduce_mem_usage)
dt_col = "date"
#data = add_time_features(data, dt_col).pipe(reduce_mem_usage)
data = data.sort_values("date")

print("start date:", data[dt_col].min())
print("end date:", data[dt_col].max())
print("data shape:", data.shape)

start date: 2015-04-22 00:00:00
end date: 2016-05-22 00:00:00
data shape: (415659, 64)


In [27]:
data.tail(40)

id          item_id      dept_id  \
414997  HOUSEHOLD_1_395_WI_3_validation  HOUSEHOLD_1_395  HOUSEHOLD_1   
414998  HOUSEHOLD_1_396_WI_3_validation  HOUSEHOLD_1_396  HOUSEHOLD_1   
414999  HOUSEHOLD_1_397_WI_3_validation  HOUSEHOLD_1_397  HOUSEHOLD_1   
415000  HOUSEHOLD_1_398_WI_3_validation  HOUSEHOLD_1_398  HOUSEHOLD_1   
415001  HOUSEHOLD_1_399_WI_3_validation  HOUSEHOLD_1_399  HOUSEHOLD_1   
414972  HOUSEHOLD_1_368_WI_3_validation  HOUSEHOLD_1_368  HOUSEHOLD_1   
414938  HOUSEHOLD_1_333_WI_3_validation  HOUSEHOLD_1_333  HOUSEHOLD_1   
414971  HOUSEHOLD_1_367_WI_3_validation  HOUSEHOLD_1_367  HOUSEHOLD_1   
414969  HOUSEHOLD_1_365_WI_3_validation  HOUSEHOLD_1_365  HOUSEHOLD_1   
414940  HOUSEHOLD_1_335_WI_3_validation  HOUSEHOLD_1_335  HOUSEHOLD_1   
414941  HOUSEHOLD_1_336_WI_3_validation  HOUSEHOLD_1_336  HOUSEHOLD_1   
414942  HOUSEHOLD_1_337_WI_3_validation  HOUSEHOLD_1_337  HOUSEHOLD_1   
414943  HOUSEHOLD_1_338_WI_3_validation  HOUSEHOLD_1_338  HOUSEHOLD_1   
414944  HOUSEHOLD_1_339_WI_3_validation  HOUSEHOLD_1_339  HOUSEHOLD_1   
414945  HOUSEHOLD_1_340_WI_3_validation  HOUSEHOLD_1_340  HOUSEHOLD_1   
414946  HOUSEHOLD_1_341_WI_3_validation  HOUSEHOLD_1_341  HOUSEHOLD_1   
414947  HOUSEHOLD_1_342_WI_3_validation  HOUSEHOLD_1_342  HOUSEHOLD_1   
414948  HOUSEHOLD_1_343_WI_3_validation  HOUSEHOLD_1_343  HOUSEHOLD_1   
414949  HOUSEHOLD_1_344_WI_3_validation  HOUSEHOLD_1_344  HOUSEHOLD_1   
414950  HOUSEHOLD_1_345_WI_3_validation  HOUSEHOLD_1_345  HOUSEHOLD_1   
414951  HOUSEHOLD_1_346_WI_3_validation  HOUSEHOLD_1_346  HOUSEHOLD_1   
414952  HOUSEHOLD_1_347_WI_3_validation  HOUSEHOLD_1_347  HOUSEHOLD_1   
414953  HOUSEHOLD_1_348_WI_3_validation  HOUSEHOLD_1_348  HOUSEHOLD_1   
414954  HOUSEHOLD_1_349_WI_3_validation  HOUSEHOLD_1_349  HOUSEHOLD_1   
414955  HOUSEHOLD_1_350_WI_3_validation  HOUSEHOLD_1_350  HOUSEHOLD_1   
414956  HOUSEHOLD_1_351_WI_3_validation  HOUSEHOLD_1_351  HOUSEHOLD_1   
414957  HOUSEHOLD_1_353_WI_3_validation  HOUSEHOLD_1_353  HOUSEHOLD_1   
414958  HOUSEHOLD_1_354_WI_3_validation  HOUSEHOLD_1_354  HOUSEHOLD_1   
414959  HOUSEHOLD_1_355_WI_3_validation  HOUSEHOLD_1_355  HOUSEHOLD_1   
414960  HOUSEHOLD_1_356_WI_3_validation  HOUSEHOLD_1_356  HOUSEHOLD_1   
414961  HOUSEHOLD_1_357_WI_3_validation  HOUSEHOLD_1_357  HOUSEHOLD_1   
414962  HOUSEHOLD_1_358_WI_3_validation  HOUSEHOLD_1_358  HOUSEHOLD_1   
414963  HOUSEHOLD_1_359_WI_3_validation  HOUSEHOLD_1_359  HOUSEHOLD_1   
414964  HOUSEHOLD_1_360_WI_3_validation  HOUSEHOLD_1_360  HOUSEHOLD_1   
414965  HOUSEHOLD_1_361_WI_3_validation  HOUSEHOLD_1_361  HOUSEHOLD_1   
414966  HOUSEHOLD_1_362_WI_3_validation  HOUSEHOLD_1_362  HOUSEHOLD_1   
414967  HOUSEHOLD_1_363_WI_3_validation  HOUSEHOLD_1_363  HOUSEHOLD_1   
414968  HOUSEHOLD_1_364_WI_3_validation  HOUSEHOLD_1_364  HOUSEHOLD_1   
414970  HOUSEHOLD_1_366_WI_3_validation  HOUSEHOLD_1_366  HOUSEHOLD_1   
415658  HOUSEHOLD_2_516_WI_3_validation  HOUSEHOLD_2_516  HOUSEHOLD_2   

           cat_id store_id state_id     d  demand        part  wm_yr_wk  \
414997  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414998  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414999  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
415000  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
415001  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414972  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414938  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414971  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414969  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414940  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414941  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414942  HOUSEHOLD     WI_3       WI  1941       0  validation     11617   
414943  HOUSEHOLD     WI_3       WI  1941       0  validation     11

# RANDOM FOREST


### 1.Creación de Matriz de Datos

In [0]:
#Importación de librerías necesarias
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

# A continuación se genera un codigo numérico para las variables: "item_id", "dept_id", "cat_id", "store_id","state_id"
from sklearn import preprocessing, metrics
def encode_categorical(df, cols):
    for col in cols:
        # Leave NaN as it is.
        le = preprocessing.LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index)
    return df
data = encode_categorical(data, ["item_id", "dept_id", "cat_id", "store_id","state_id",'wm_yr_wk'])
#A continuación se eliminan columnas que no van a ser utilizadas en el modelo
data=data.drop(columns=['part','d'])


In [0]:
#A continuación se eliminan los n/a
data = data.fillna(0)

In [30]:
data.tail()

id  item_id  dept_id  cat_id  store_id  \
414966  HOUSEHOLD_1_362_WI_3_validation      354        0       0         0   
414967  HOUSEHOLD_1_363_WI_3_validation      355        0       0         0   
414968  HOUSEHOLD_1_364_WI_3_validation      356        0       0         0   
414970  HOUSEHOLD_1_366_WI_3_validation      358        0       0         0   
415658  HOUSEHOLD_2_516_WI_3_validation     1046        1       0         0   

        state_id  demand  wm_yr_wk       date  year_2011  year_2012  \
414966         0       0        57 2016-05-22          0          0   
414967         0       0        57 2016-05-22          0          0   
414968         0       0        57 2016-05-22          0          0   
414970         0       0        57 2016-05-22          0          0   
415658         0       0        57 2016-05-22          0          0   

        year_2013  year_2014  year_2015  year_2016  quarter_1  quarter_2  \
414966          0          0          0          1          0          1   
414967          0          0          0          1          0          1   
414968          0          0          0          1          0          1   
414970          0          0          0          1          0          1   
415658          0          0          0          1          0          1   

        quarter_3  quarter_4  01_January  02_February  03_March  04_April  \
414966          0          0           0            0         0         0   
414967          0          0           0            0         0         0   
414968          0          0           0            0         0         0   
414970          0          0           0            0         0         0   
415658          0          0           0            0         0         0   

        05_May  06_June  07_July  08_August  09_September  10_October  \
414966       1        0        0          0             0           0   
414967       1        0        0          0             0           0   
414968       1        0        0          0             0           0   
414970       1        0        0          0             0           0   
415658       1        0        0          0             0           0   

        11_November  12_December  01_Monday  02_Tuesday  03_Wednesday  \
414966            0            0          0           0             0   
414967            0            0          0           0             0   
414968            0            0          0           0             0   
414970            0            0          0           0             0   
415658            0            0          0           0             0   

        04_Thursday  05_Friday  06_Saturday  07_Sunday  is_weekend EventoHoy  \
414966            0          0            0          1           1         0   
414967            0          0            0          1           1         0   
414968            0          0            0          1           1         0   
414970            0          0            0          1           1         0   
415658            0          0            0          1           1         0   

       EventoMañana EventoPasadoMañana  event_type_Bool_Cultural  \
414966            0                  0                         0   
414967            0                  0                         0   
414968            0                  0                         0   
414970            0                  0                         0   
415658            0                  0                         0   

        event_type_Bool_National  event_type_Bool_Religious  \
414966                         0                          0   
414967                         0                          0   
414968                         0                          0   
414970                         0                          0   
415658                         0                          0   

        event_type_Bool_Sporting  sell_price  rolling_std_t7  rollin

### 2.Creación del modelo 

In [0]:
#Se definen los datos de Entrenamiento
x_train = data[data['date'] <= '2016-03-30']
y_train = x_train['demand']
#Se definen los datos de Validación
x_val = data[(data['date'] > '2016-03-30') & (data['date'] <= '2016-04-24')]
y_val = x_val['demand']
dias_val=x_val['date']

#Se definen los datos de testeo
test = data[(data['date'] >= '2016-03-01')]

#A continuación se elimina la columna demanda y fecha
x_train=x_train.drop(columns=['date','demand','id'])
x_val=x_val.drop(columns=['date','demand','id'])

In [32]:
print(test.shape)
print(dias_val.shape)
print(x_val.shape)

(86901, 62)
(26175,)
(26175, 59)


In [33]:
test.head()

id  item_id  dept_id  cat_id  store_id  \
329458  HOUSEHOLD_2_169_WI_3_validation      700        1       0         0   
329457  HOUSEHOLD_2_168_WI_3_validation      699        1       0         0   
329456  HOUSEHOLD_2_167_WI_3_validation      698        1       0         0   
329455  HOUSEHOLD_2_166_WI_3_validation      697        1       0         0   
329454  HOUSEHOLD_2_165_WI_3_validation      696        1       0         0   

        state_id  demand  wm_yr_wk       date  year_2011  year_2012  \
329458         0       0        45 2016-03-01          0          0   
329457         0       0        45 2016-03-01          0          0   
329456         0       0        45 2016-03-01          0          0   
329455         0       0        45 2016-03-01          0          0   
329454         0       0        45 2016-03-01          0          0   

        year_2013  year_2014  year_2015  year_2016  quarter_1  quarter_2  \
329458          0          0          0          1          1          0   
329457          0          0          0          1          1          0   
329456          0          0          0          1          1          0   
329455          0          0          0          1          1          0   
329454          0          0          0          1          1          0   

        quarter_3  quarter_4  01_January  02_February  03_March  04_April  \
329458          0          0           0            0         1         0   
329457          0          0           0            0         1         0   
329456          0          0           0            0         1         0   
329455          0          0           0            0         1         0   
329454          0          0           0            0         1         0   

        05_May  06_June  07_July  08_August  09_September  10_October  \
329458       0        0        0          0             0           0   
329457       0        0        0          0             0           0   
329456       0        0        0          0             0           0   
329455       0        0        0          0             0           0   
329454       0        0        0          0             0           0   

        11_November  12_December  01_Monday  02_Tuesday  03_Wednesday  \
329458            0            0          0           1             0   
329457            0            0          0           1             0   
329456            0            0          0           1             0   
329455            0            0          0           1             0   
329454            0            0          0           1             0   

        04_Thursday  05_Friday  06_Saturday  07_Sunday  is_weekend EventoHoy  \
329458            0          0            0          0           0         0   
329457            0          0            0          0           0         0   
329456            0          0            0          0           0         0   
329455            0          0            0          0           0         0   
329454            0          0            0          0           0         0   

       EventoMañana EventoPasadoMañana  event_type_Bool_Cultural  \
329458            0                  0                         0   
329457            0                  0                         0   
329456            0                  0                         0   
329455            0                  0                         0   
329454            0                  0                         0   

        event_type_Bool_National  event_type_Bool_Religious  \
329458                         0                          0   
329457                         0                          0   
329456                         0                          0   
329455                         0                          0   
329454                         0                          0   

        event_type_Bool_Sporting  sell_price  rolling_std_t7  rollin

In [0]:
def MASE(training_series, testing_series, prediction_series):
    """
    Computes the MEAN-ABSOLUTE SCALED ERROR forcast error for univariate time series prediction.
    
    See "Another look at measures of forecast accuracy", Rob J Hyndman
    
    parameters:
        training_series: the series used to train the model, 1d numpy array
        testing_series: the test series to predict, 1d numpy array or float
        prediction_series: the prediction of testing_series, 1d numpy array (same size as testing_series) or float
        absolute: "squares" to use sum of squares and root the result, "absolute" to use absolute values.
    
    """
    #print "Needs to be tested."
    n = training_series.shape[0]
    d = np.abs(  np.diff( training_series) ).sum()/(n-1)
    
    errors = np.abs(testing_series - prediction_series )
    return errors.mean()/d

In [0]:
# Se genera el modelo random forest base con los parámetros que vienen por defecto en la librería
ModelRF_base = RandomForestRegressor(random_state=123).fit(x_train, y_train)

# Predicción 
y_pred=ModelRF_base.predict(x_val)


In [36]:
#A continuación se lleva a cabo una evaluación por medio de las métricas de Accurancy y MSE para éste primer modelo

#Evaluación del Accuracy
ModelRF_base_Score = ModelRF_base.score(x_val, y_val)

#Evaluación de MSE
ModelRF_base_MSE= mean_squared_error(y_val, y_pred)

#Evaluación de MASE
ModelRF_base_MASE = MASE(y_train, y_val, y_pred)

#Imprimen los resultados
print('Random Forest Accuracy: {}'.format(ModelRF_base_Score))
print('Random Forest MSE: {}'.format(ModelRF_base_MSE))
print('Random Forest NMSE: {}'.format(ModelRF_base_MASE))

Random Forest Accuracy: 0.5212949128726682
Random Forest MSE: 1.7075624336198663
Random Forest NMSE: 0.6524565153351786


### 3.Identificación de variables relevantes

In [37]:
ModelRF_base.feature_importances_
imp_feat=pd.Series(ModelRF_base.feature_importances_,index=x_train.columns.tolist())
imp_feat.sort_values(ascending=False)

rolling_mean_t7              0.363925
rolling_mean_t30             0.144795
item_id                      0.057167
sell_price                   0.047310
wm_yr_wk                     0.042525
rolling_std_t7               0.032763
rolling_std_t30              0.031139
rolling_std_t60              0.028753
rolling_mean_t60             0.028204
rolling_kurt_t30             0.027553
rolling_skew_t30             0.027365
rolling_std_t90              0.024407
rolling_mean_t90             0.021176
rolling_std_t180             0.014781
rolling_mean_t180            0.014398
is_weekend                   0.012558
05_Friday                    0.007456
01_Monday                    0.005806
02_Tuesday                   0.005553
04_Thursday                  0.005372
03_Wednesday                 0.005369
06_Saturday                  0.005038
EventoMañana                 0.005013
EventoPasadoMañana           0.004732
07_Sunday                    0.003916
rolling_price_std_t30        0.003087
EventoHoy   

In [0]:
#A continuación se limitan los datos de entrenamiento sólo a las variables mas importantes

features = imp_feat[imp_feat >= 0.008] 

# Se ajustan los datos de entrenamiento solo con las variables seleccionadas

x_train=x_train[features.index]
x_val=x_val[features.index]
x_test=test[features.index]

##4.Análisis Sensibilidad

In [0]:
#Se define función para evaluar rendimiento de los modelos
from sklearn.metrics import mean_squared_error, r2_score

def evaluate(model, x_val, y_val):
    y_pred=model.predict(x_val)
    MSE = mean_squared_error(y_val, y_pred)
    r2 =  r2_score(y_val, y_pred)

    print('Rendimiento del modelo')
    print('Random Forest MSE: {}'.format(MSE))
    print('Random Forest R^2: {}'.format(r2))

A continuación se va a proceder a ajustar el modelo encontrado por medio de random search, en esta fase lo que se pretende es evaluar ya no combinaciones aleatorias si no todas las combinaciones próximas a los parámetros encontrados por random search

## 4.Creación del modelo Final

In [40]:
from sklearn.model_selection import GridSearchCV

ModelRF_final = RandomForestRegressor(bootstrap=True,max_depth=10,max_features=10,min_samples_split=9,
                                      min_samples_leaf=4,n_estimators=50,random_state=0).fit(x_train,y_train)

evaluate(ModelRF_final,x_val,y_val)

Rendimiento del modelo
Random Forest MSE: 1.680260764247111
Random Forest R^2: 0.5289487753367775


In [0]:
y_pred_final=ModelRF_final.predict(x_test)

In [0]:
#plt.plot(dias_val, y_pred_final,'bo', label='Predicción',)
#plt.plot(dias_val, y_val,'ro', label='Real',)
#plt.legend()
#plt.title("Predicción vs Actual")

##7.Generación de Archivo Submission


In [0]:
test['demand'] = y_pred_final

In [44]:
test

id  item_id  dept_id  cat_id  store_id  \
329458  HOUSEHOLD_2_169_WI_3_validation      700        1       0         0   
329457  HOUSEHOLD_2_168_WI_3_validation      699        1       0         0   
329456  HOUSEHOLD_2_167_WI_3_validation      698        1       0         0   
329455  HOUSEHOLD_2_166_WI_3_validation      697        1       0         0   
329454  HOUSEHOLD_2_165_WI_3_validation      696        1       0         0   
...                                 ...      ...      ...     ...       ...   
414966  HOUSEHOLD_1_362_WI_3_validation      354        0       0         0   
414967  HOUSEHOLD_1_363_WI_3_validation      355        0       0         0   
414968  HOUSEHOLD_1_364_WI_3_validation      356        0       0         0   
414970  HOUSEHOLD_1_366_WI_3_validation      358        0       0         0   
415658  HOUSEHOLD_2_516_WI_3_validation     1046        1       0         0   

        state_id    demand  wm_yr_wk       date  year_2011  year_2012  \
329458         0  0.105351        45 2016-03-01          0          0   
329457         0  0.143839        45 2016-03-01          0          0   
329456         0  0.170359        45 2016-03-01          0          0   
329455         0  0.157733        45 2016-03-01          0          0   
329454         0  0.148668        45 2016-03-01          0          0   
...          ...       ...       ...        ...        ...        ...   
414966         0  0.289989        57 2016-05-22          0          0   
414967         0  0.550195        57 2016-05-22          0          0   
414968         0  0.185457        57 2016-05-22          0          0   
414970         0  0.481900        57 2016-05-22          0          0   
415658         0  0.093094        57 2016-05-22          0          0   

        year_2013  year_2014  year_2015  year_2016  quarter_1  quarter_2  \
329458          0          0          0          1          1          0   
329457          0          0          0          1          1          0   
329456          0          0          0          1          1          0   
329455          0          0          0          1          1          0   
329454          0          0          0          1          1          0   
...           ...        ...        ...        ...        ...        ...   
414966          0          0          0          1          0          1   
414967          0          0          0          1          0          1   
414968          0          0          0          1          0          1   
414970          0          0          0          1          0          1   
415658          0          0          0          1          0          1   

        quarter_3  quarter_4  01_January  02_February  03_March  04_April  \
329458          0          0           0            0         1         0   
329457          0          0           0            0         1         0   
329456          0          0           0            0         1         0   
329455          0          0           0            0         1         0   
329454          0          0           0            0         1         0   
...           ...        ...         ...          ...       ...       ...   
414966          0          0           0            0         0         0   
414967          0          0           0            0         0         0   
414968          0          0           0            0         0         0   
414970          0          0           0            0         0         0   
415658          0          0           0            0         0         0   

        05_May  06_June  07_July  08_August  09_September  10_October  \
329458       0        0        0          0             0           0   
329457       0        0        0          0             0           0   
329456       0        0        0          0             0           0   
329455       0        0        0          0             0           0   
329454  

In [0]:
from google.colab import files
predictions = test[['id', 'date', 'demand']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(83)]

evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation = submission[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])
final.to_csv('submission.csv', index = False)
#files.download('submission.csv')


In [46]:
final

id        F1        F2        F3        F4  \
0     HOUSEHOLD_1_001_WI_3_validation  0.426198  0.401523  0.420501  0.414364   
1     HOUSEHOLD_1_002_WI_3_validation  0.388829  0.388829  0.338994  0.322760   
2     HOUSEHOLD_1_003_WI_3_validation  0.488662  0.459437  0.449818  0.474174   
3     HOUSEHOLD_1_004_WI_3_validation  3.341797  3.417514  3.553888  3.234263   
4     HOUSEHOLD_1_005_WI_3_validation  2.441996  2.391550  2.110021  2.038517   
...                               ...       ...       ...       ...       ...   
1042  HOUSEHOLD_2_512_WI_3_validation  0.229811  0.227434  0.203072  0.203072   
1043  HOUSEHOLD_2_513_WI_3_validation  0.314042  0.313443  0.305878  0.300514   
1044  HOUSEHOLD_2_514_WI_3_validation  0.093094  0.093094  0.093094  0.093094   
1045  HOUSEHOLD_2_515_WI_3_validation  0.260761  0.254999  0.254999  0.254999   
1046  HOUSEHOLD_2_516_WI_3_validation  0.125476  0.120234  0.117339  0.123457   

            F5        F6        F7        F8        F9       F10       F11  \
0     0.468353  0.478367  0.400174  0.388926  0.388926  0.386103  0.386103   
1     0.337802  0.337802  0.285113  0.285113  0.303703  0.303703  0.303703   
2     0.573396  0.502546  0.457915  0.457915  0.457915  0.433747  0.496515   
3     4.142218  4.369140  3.062626  2.848308  3.064418  3.179957  3.168474   
4     2.364476  2.431349  1.684433  1.549307  1.831547  1.708661  1.808559   
...        ...       ...       ...       ...       ...       ...       ...   
1042  0.197130  0.234640  0.223988  0.277650  0.277650  0.277650  0.277650   
1043  0.306032  0.306032  0.314965  0.312304  0.317459  0.317908  0.317908   
1044  0.093094  0.093094  0.093094  0.093094  0.093094  0.093094  0.093887   
1045  0.272014  0.272014  0.254999  0.254999  0.254999  0.254999  0.254999   
1046  0.124212  0.119054  0.119054  0.119054  0.119054  0.119054  0.111953   

           F12       F13       F14       F15       F16       F17       F18  \
0     0.498114  0.498114  0.420967  0.428375  0.427761  0.419841  0.427936   
1     0.330295  0.317557  0.275456  0.275456  0.265393  0.265393  0.284961   
2     0.564547  0.564547  0.441984  0.444539  0.444539  0.459280  0.432343   
3     4.582105  3.796131  2.271031  2.125192  2.118178  2.155904  2.088268   
4     2.424523  2.312326  1.466741  1.670390  1.407302  1.456717  1.383411   
...        ...       ...       ...       ...       ...       ...       ...   
1042  0.329921  0.415580  0.355536  0.283202  0.253154  0.253154  0.253154   
1043  0.335782  0.335782  0.317174  0.311675  0.325675  0.321319  0.337699   
1044  0.093887  0.093887  0.093887  0.093887  0.093887  0.093887  0.093887   
1045  0.254641  0.223666  0.220428  0.207008  0.209057  0.209057  0.227664   
1046  0.108479  0.108479  0.108479  0.106998  0.106998  0.106998  0.106998   

           F19       F20       F21       F22       F23       F24       F25  \
0     0.481433  0.485393  0.407758  0.412326  0.407758  0.399848  0.373023   
1     0.328225  0.327063  0.329839  0.329839  0.329839  0.326437  0.292808   
2     0.733551  0.541595  0.530087  0.545432  0.556584  0.545432  0.448295   
3     3.157361  2.761696  2.560373  2.404332  2.490351  2.271938  2.200128   
4     2.368362  2.353867  1.243437  1.210374  1.266123  1.369433  1.369433   
...        ...       ...       ...       ...       ...       ...       ...   
1042  0.323907  0.315965  0.331080  0.344432  0.331508  0.331508  0.317983   
1043  0.491121  0.571326  0.585634  0.585634  0.500161  0.612549  0.500161   
1044  0.093887  0.093887  0.093887  0.093887  0.093887  0.093887  0.093887   
1045  0.235366  0.248898  0.257636  0.254975  0.254975  0.254975  0.242742   
1046  0.106998  0.106998  0.106998  0.106998  0.106998  0.106998  0.106998   

           F26       F27       F28       F29       F30       F31       F32  \
0     0.428374  0.499875  0.422359  0.417855  0.414742  0.414742  0.409245   
1     0.317557  0.300342  0.302314  0.345965  0.322559  0.322559  0.322559   
2     0.541327  0

In [0]:
#Fuente https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

#Fuente http://dkopczyk.quantee.co.uk/hyperparameter-optimization/ 